In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer


In [3]:
import torch

In [4]:
device = torch.get_default_device()
torch.set_default_device(device)

In [5]:

model_name = "Qwen/Qwen3-0.6B"

# load the tokenizer and the model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
)


In [9]:
# prepare the model input
prompt = """
SYSTEM / INSTRUCTIONS FOR THE MODEL:
You are an expert resume writer and career coach specialized in rewriting candidate experiences to match job descriptions.  
You will be given an array of items. Each item contains:
  1) "jd_snippet": a short text span taken from a job description (responsibility, role, or technology).
  2) "candidate_experiences": a (possibly empty) list of the candidate's past experience items relevant to that jd_snippet. Each candidate experience is a short paragraph describing what they did, technologies used, and any achievements (may or may not include quantitative impact).

Your task: For each item, produce a set of resume-ready experience lines that can be directly added to the candidate's CV. Use the candidate_experiences when available to create factual, tailored bullets. If candidate_experiences is empty, generate plausible, conservative, and honest-sounding suggested bullets aligned to the jd_snippet (mark them as suggestions). When a quantitative impact is missing in a source experience, the model should **propose** a realistic, conservative numeric impact and mark that number as suggested.

OUTPUT FORMAT:
Output must be exactly one JSON object following this schema. Do not output anything else (no commentary, no markdown, no extra keys).

Root object schema:
{
  "generated_at": "<ISO8601 timestamp>",
  "job_customizations": [
    {
      "jd_snippet": "<original job description snippet>",
      "matches": [
        {
          "source_experience": "<original candidate experience text or empty string if none>",
          "resume_bullet": "<single resume bullet — 1-2 sentences, action-first, past tense for prior roles, present for current role>",
          "skills": ["comma-separated skill tags"],
          "impact": {
            "value": "<numeric value or string with units, e.g. '25%' or 'reduced latency by 300ms' or null>",
            "is_suggested": <true|false>,
            "explain": "<1-2 sentence note about how the impact was obtained or why it was suggested>"
          },
          "confidence": "<HIGH | MEDIUM | LOW>",
          "rationale": "<brief 1-sentence mapping explaining which parts of the jd_snippet and source_experience were used to create this bullet>"
        }
        /* up to 3 matches per jd_snippet */
      ],
      "notes": "<optional short note if any special assumptions were made; empty string if none>"
    }
    /* one object per jd_snippet in input */
  ],
  "warnings": ["..."]   /* array of strings; e.g. if many suggestions used, warn reviewer */
}

OUTPUT RULES (strict):
1. Provide up to 3 resume bullets (matches) per jd_snippet. Prefer 1–2 strong bullets.
2. Bullets must be 8–22 words ideally; at most 2 short sentences.
3. Start each bullet with a strong action verb (Design, Implement, Led, Built, Optimized, Reduced, Launched, etc.).
4. Always fill "skills" with explicit normalized tokens detected or implied (e.g., ["Python","FastAPI","Kafka"]). If none, return an empty array.
5. If a numeric impact is present in the source experience, copy it verbatim and set "is_suggested": false.
6. If no numeric impact exists but you add one, choose a conservative, realistic estimate, set "is_suggested": true, and explain why in "impact.explain".
7. If candidate_experiences is empty, create suggested bullets, set "source_experience" to empty string, set "impact.is_suggested" true for any numeric figures, and mark "confidence" as MEDIUM or LOW.
8. Set "confidence" to HIGH when the resume_bullet closely mirrors explicit content in candidate_experiences or when concrete numbers are present; MEDIUM when derived with some inference; LOW when mostly invented or inferred from job text only.
9. Do NOT hallucinate specifics like company names, dates, team sizes, or proprietary metrics. If you must suggest numbers, make them conservative and explain them in "impact.explain".
10. The JSON must be valid. Use ISO8601 for "generated_at" (e.g., "2025-12-22T15:04:05Z").
11. Put any extra assumptions in the "notes" field for that jd_snippet.
12. Provide a "warnings" array at root level. If no warnings, use an empty array.

EXAMPLES (few-shot). Input is an array of items. The model should use these examples as guidance for formatting and style.

EXAMPLE INPUT 1:
[
  {
    "jd_snippet": "Design and implement microservices in Python.",
    "candidate_experiences": [
      "Built REST APIs for a billing platform using Python, Flask and PostgreSQL; improved request throughput.",
      "Implemented microservices for payment processing using Python and Docker; no metrics recorded."
    ]
  }
]

EXPECTED OUTPUT 1 (JSON only):
{
  "generated_at": "2025-12-22T00:00:00Z",
  "job_customizations": [
    {
      "jd_snippet": "Design and implement microservices in Python.",
      "matches": [
        {
          "source_experience": "Built REST APIs for a billing platform using Python, Flask and PostgreSQL; improved request throughput.",
          "resume_bullet": "Designed and implemented RESTful microservices in Python (Flask) for billing, improving throughput.",
          "skills": ["Python","Flask","PostgreSQL","Microservices"],
          "impact": {
            "value": null,
            "is_suggested": false,
            "explain": "No explicit numeric metric in source; retained qualitative improvement as-is."
          },
          "confidence": "HIGH",
          "rationale": "Used explicit Python/Flask/PostgreSQL implementation from source experience to match 'microservices in Python'."
        },
        {
          "source_experience": "Implemented microservices for payment processing using Python and Docker; no metrics recorded.",
          "resume_bullet": "Implemented payment-processing microservices in Python and Docker, enabling modular deployment.",
          "skills": ["Python","Docker","Microservices"],
          "impact": {
            "value": null,
            "is_suggested": false,
            "explain": "Source included no numeric impact; preserved factual implementation details."
          },
          "confidence": "MEDIUM",
          "rationale": "Direct mapping from source; job snippet requested microservices in Python so tech aligns."
        }
      ],
      "notes": ""
    }
  ],
  "warnings": []
}

EXAMPLE INPUT 2 (no relevant candidate experience):
[
  {
    "jd_snippet": "Build high-throughput data pipelines using Kafka and Spark.",
    "candidate_experiences": []
  }
]

EXPECTED OUTPUT 2:
{
  "generated_at": "2025-12-22T00:00:00Z",
  "job_customizations": [
    {
      "jd_snippet": "Build high-throughput data pipelines using Kafka and Spark.",
      "matches": [
        {
          "source_experience": "",
          "resume_bullet": "Designed and implemented high-throughput data pipelines using Kafka and Spark to process streaming events.",
          "skills": ["Kafka","Spark","Data Pipelines"],
          "impact": {
            "value": "processed 10M events/day",
            "is_suggested": true,
            "explain": "No candidate experience provided; suggested a conservative throughput estimate as placeholder—please adjust based on real numbers."
          },
          "confidence": "LOW",
          "rationale": "Generated suggested bullet from JD snippet; no candidate experience available to ground the claim."
        }
      ],
      "notes": "Generated suggestion — confirm throughput number with candidate before using."
    }
  ],
  "warnings": ["Multiple suggested impacts inserted; verify suggested numeric values with the candidate."]
}

---

INPUT:
{
  "jd_snippet": "We are seeking a Senior Backend Engineer. Responsibilities: Design and implement microservices in Python. Build high-throughput data pipelines with Kafka and Spark. Required: Bachelor's degree or equivalent.",
  "candidate_experiences": [
    "Built REST APIs and microservices in Python and Flask; improved request throughput by optimizing database queries.",
    "Implemented ETL jobs using Spark to transform event logs into analytics tables.",
    "Managed internships and helped onboard two junior engineers."
  ]
}

Generate Output!!!
"""


In [10]:
import math
import time

messages = [
    {"role": "user", "content": prompt}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True,
    enable_thinking=True # Switches between thinking and non-thinking modes. Default is True.
)
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)


start_time = time.time()
# conduct text completion
generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=32768
)

end_time = time.time()

output_ids = generated_ids[0][len(model_inputs.input_ids[0]):].tolist() 

# parsing thinking content
try:
    # rindex finding 151668 (</think>)
    index = len(output_ids) - output_ids[::-1].index(151668)
except ValueError:
    index = 0

thinking_content = tokenizer.decode(output_ids[:index], skip_special_tokens=True).strip("\n")
content = tokenizer.decode(output_ids[index:], skip_special_tokens=True).strip("\n")

print("thinking content:", thinking_content)
print("content:", content)

elapsed_time = end_time - start_time
elapsed_seconds = math.ceil(elapsed_time)
print(f"Elapsed Time: {elapsed_seconds // 60}:{elapsed_seconds % 60}")



thinking content: <think>
Okay, let's tackle this problem. The user provided an array of items with job descriptions and candidate experiences, and I need to generate resume-ready experience lines. Let me start by understanding each element.

First, the job description snippet is: "We are seeking a Senior Backend Engineer. Responsibilities: Design and implement microservices in Python. Build high-throughput data pipelines with Kafka and Spark. Required: Bachelor's degree or equivalent." So the job has two main responsibilities: designing and implementing microservices in Python, and building high-throughput data pipelines with Kafka and Spark.

The candidate experiences are three items. Let me look through them. The first candidate experience mentions building REST APIs and microservices in Python and Flask, improving request throughput. That's a good start. The second one is about implementing ETL jobs using Spark, transforming event logs into analytics tables. The third is managing i